## TOÁN ỨNG DỤNG VÀ THỐNG KÊ (MTH00051)

### I. Thông tin
- **Môn học:** Toán ứng dụng và thống kê
- **Lớp:** 22_2
- **Họ và tên:** Nguyễn Trung Quốc  
- **MSSV:** 22120301

### II. Bài tập

**Đề bài:** Trình bày giải thuật giải hệ phương trình tuyến tính Ax = b với phép khử Gauss và phép thế ngược.

1. **Phép khử Gauss (Gauss Elimination)**
    - **Yêu cầu:**
        - **Input:** Ma trận A và vector b
        - **Output:** Ma trận bậc thang có được từ ma trận A
    - **Giải thuật:**
        - Bước 1: Bắt đầu với cột 1, tráo đổi với dòng khác để chọn "giá trị xoay" có trị tuyệt đối lớn nhất. Nếu tất cả đều bằng 0, chuyển sang cột tiếp theo bên phải.
        - Bước 2: Sử dụng dòng 1 để khử các giá trị dưới ở cột 1 thành 0, bằng cách sử dụng phép biển đổi cơ sở dòng.
        - Bước 3: Lặp lại bước 1 và 2 cho cột tiếp theo (2, 3, ...)

2. **Phép thế ngược (Back-substitution)**
    - **Yêu cầu:**
        - **Input:** A là ma trận có dạng bậc thang từ ma trận mở rộng của hệ phương trình
        - **Output:** Nghiệm của phương trình (trường hợp có duy nhất một hoặc vô số nghiệm), nếu vô nghiệm cần phải thông báo
    - **Giải thuật:**
        - Bước 1: Xác định trường hợp có nghiệm và vô nghiệm 
            + Trường hợp rank(A) < rank(A|b): thông báo vô nghiệm
            + Trường hợp rank(A) = rank(A|b) < n = số ẩn: thông báo vô số nghiệm
            + Trường hợp rank(A) = rank(A|b) = n = số ẩn: thông báo có 1 nghiệm duy nhất
        - Bước 2: Với trường hợp có nghiệm, dùng 1 mảng để biểu diễn Xi. Ví dụ: X1 = [1, 2, 3, 4] thì sẽ có dạng X1 = 1 + 2*a + 3*b + 4*c.
        - Bước 3: Tìm các cột có giá trị xoay khác 0 sẽ không phải là ẩn tự do. Các cột còn lại tương ứng với các ẩn tự do.
        - Bước 4: Lần lượt dùng phép thế ngược từ dưới lên để tìm Xi (với các ẩn tự do) bằng các phép toán trên mảng.


In [19]:
def Gauss_elimination(A, B):
    n = len(A)
    cur_col = 0
    
    for i in range(n):
        #Finding pivot: which have largest absolute
        max_index = i
        for j in range (i+1, n):
            if abs(A[j][cur_col]) > abs(A[max_index][cur_col]):
                max_index = j
        A[i], A[max_index] = A[max_index], A[i]
        B[i], B[max_index] = B[max_index], B[i]
        
        #Finding cur_col if A[i][i] == 0
        while A[i][cur_col] == 0 and cur_col < n-1: 
            cur_col += 1
            max_index = i
            for j in range (i+1, n):
                if abs(A[j][cur_col]) > abs(A[max_index][cur_col]):
                    max_index = j
            A[i], A[max_index] = A[max_index], A[i]
            B[i], B[max_index] = B[max_index], B[i]
        if A[i][cur_col] == 0: continue 

        #Subtracting when knowing cur_col
        for j in range(i+1 , n):
            factor = A[j][cur_col] / A[i][cur_col]
            for k in range(i , n):
                A[j][k] -= factor * A[i][k]    
            B[j] -= factor * B[i]          
        if cur_col < n-1: cur_col += 1
    
    #Round number to remove error when using float 
    A = [[round(num, 6) for num in sublist] for sublist in A]
    B = [round(num, 6) for num in B] 
    return (A, B)

In [20]:
def count_zero(A):
    '''
    Count number of zero_line from bottom 
    
    A -> coef of matrix
    
    Return number of zero_lines
    '''
    
    res = 0
    for a in A:
        if len(a) == a.count(0): 
            res += 1
    return res            


def compute_operator(a , type, X = [], Y = []):
    '''
    Computing operator + - /
    
    a -> coef 
    type -> 0, 1, 2
    X -> list1 
    Y -> list2   
    
    Return the result 
    '''
    res = []
    if type == 0:
        res = [X[i] + a*Y[i] for i in range(len(X))] # operator +
    if type == 1:
        res = [X[i] - a*Y[i] for i in range(len(X))] # operator -
    if type == 2:
        res = [X[i]/a for i in range(len(X))]   # operator /
    
    return res

def print_solution(X):
    '''
    Print solution
    
    X -> matrix like 
    [[2 , a , b ],
     [3 , a , b ],
     [4 , a , b]]
       
    '''   

    start_ch = 'a'
    for i in range(len(X)):
        c = X[i][0] 
        if c == 0: c = ''
        print(f'x{i+1} = {X[i][0]}', end =' ')
        for j in range(len(X[i])):
            if j > 0:
                ch = chr(ord(start_ch) + j-1)
                if X[i][j] > 0: 
                    print(f'+ {X[i][j]} {ch} ', end= ' ')
                if X[i][j] < 0:
                    print(f'{X[i][j]} {ch} ', end= ' ')
        print()

In [21]:
def back_substitution(A, B):
    '''
    Find general solution if it has 1 or infinity solutions
    
    A -> coef of matrix
    B -> vector 
    
    Return general solution and Print like: 
        x1 = 0 + 1 a + 2 b + 3 c - 4 d...
        x2 = ...
        ...
    '''
    
    n = len(A)
    list_unknowns = []
    cur_col = 0
    #Find unknow which is not free
    for i in range(n):
        while A[i][cur_col] == 0 and cur_col < n-1: 
            cur_col += 1
        if A[i][cur_col] == 0 : 
            continue
        list_unknowns.append(cur_col)
        if cur_col < n-1: cur_col += 1
    
    #Number of free unknowns
    free_unknowns = n - len(list_unknowns) + 1 # 1 + 2*a + 3*b + 4*c...
    
    #Finding X which is solution like X = [[0 , a , b ],
    #                                      [0 , a , b ],
    #                                      [0 , a , b]]
    
    X = [[0 for _ in range(free_unknowns)] for _ in range(n)]
    unk = 1 # is pivot 
    for i in range(n-1,-1, -1):
        if i not in list_unknowns:
            X[i][unk] = 1
            unk += 1
        if i < len(list_unknowns):
            cur_col = list_unknowns[i] 
            B_tmp = [0 for _ in range(free_unknowns)]
            B_tmp[0] = B[i]
            X[cur_col] = B_tmp
            for j in range(cur_col+1, n):
                # X[i] = X[i] - A[i][j]*X[j]
                X[cur_col] = compute_operator(A[i][j], 1, X[cur_col], X[j])
            #X[i] /= A[i][cur_col]
            X[cur_col] = compute_operator(A[i][cur_col], 2, X[cur_col])
    
    #Round to remove error when using float
    X = [[round(num, 2) for num in sublist] for sublist in X]
    
    print_solution(X)
    return X

In [22]:
def solve_systems_of_equations(A , B):
    '''
    Solve systems of equasions 
    
    A -> coef of matrix
    B -> vector
    
    Print the solution 
    '''
    n = len(A)
    A, B = Gauss_elimination(A, B)

    #Computing rank and conclusion
    rankB = len(B) - B.count(0)
    rankA = len(A) - count_zero(A)
    
    if rankA == max(rankA, rankB):
        if rankA < n:
            print('He phuong trinh co vo so nghiem')
        else:
            print('He phuong trinh co 1 nghiem duy nhat')    
        back_substitution(A, B)
    else:
        print('He phuong trinh vo nghiem')

In [25]:


if __name__ == '__main__':
    #Example A and B
    # A = [[2, 1, -1],
    #      [-3, -1, 2],
    #      [-2, 1, 2]]
    # B = [8, -11, -3]


    # A = [[1,2,3,-2],
    #      [-2,1,2,3],
    #      [3,2,-1,2],
    #      [2,-3,2,1]]
    # B = [6,-8,4,-8]


    # A = [[1,1,0,-3,-1],
    #      [1,-1,2,-1,0],
    #      [4,-2,6,3,-4],
    #      [2,4,-2,4,-7],
    #      [0,0,0,0,0]]
    # B = [-2,1,7,1,0]

    A = [[1,3,-1,2,2],
         [-1,0,4,-2,1],
         [4,13,-2,3,12],
         [3,11,0,1,11],
         [1,2,-1,-4,3]]
    B = [0,-9,-6,-9,8]

    # A = [[1,3,1,-1,2],
    #      [-2,-9,-5,11,-13],
    #      [4,13,7,-7,19],
    #      [-2,-7,-2,8,-12],
    #      [0,-1,-2,-2,11]]
    # B = [12,-48,56,-47,20]

    # A = [[1,5,1,-3,4],
    #      [-2,-9,1,2,-6],
    #      [2,5,-16,14,4],
    #      [1,2,-9,9,2],
    #      [0,-2,-4,8,-5]]
    # B = [-7,1,57,34,22]

    # A = [[1,-3,-5,-2,-2],
    #      [-2,6,11,5,3],
    #      [-3,9,15,6,3],
    #      [1,-3,-10,-7,6],
    #      [-1,3,5,2,5]]
    # B = [0,0,0,0,0]

#     A = [[1,0,0,-2,-4],
#         [5,3,-3,-22,-20],
#         [2,1,-2,-8,-8],
#         [0,-3,-1,17,15],
#         [-2,-3,-1,12,-4]]
#     B = [-1,7,5,-15,5]

    solve_systems_of_equations(A , B)


He phuong trinh co vo so nghiem
x1 = -147.0 -29.0 a  
x2 = 40.0 + 7.0 a  
x3 = -43.0 -8.0 a  
x4 = -8.0 -1.0 a  
x5 = 0 + 1 a  
